# Investigating clinical notes regarding falls in older adults
Author: Wendy Phillips

## Problem statement

Falls in older adults are frequent and can have major health impacts.

## Data description

Clinical notes and metadata associated with 111,000 visits to the doctor were analyzed.

## Analytic approach

Natural Language Processing (NLP) was applied to the clinical notes to extract informative content. Specifically, the NLP tool spaCy was used to build a model that could identify and label specific spans of the text that held information of interest. 

In [ ]:
import numpy as np
import pandas as pd
import json
from pathlib import Path
import re
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from spacy.tokens import Span
from spacy.tokens import DocBin
import collections

In [ ]:
print(spacy.__version__)

### load variable mapping

In [ ]:
with Path("primary_data/variable_mapping.json").open("r") as f:
    mapping = json.load(f, parse_int=True)

In [ ]:
# convert the encoded values in the mapping to integers since they get read in as strings
for c in mapping.keys():
    mapping[c] = {int(k): v for k, v in mapping[c].items()}

### load primary data

In [ ]:
df = pd.read_csv(
    "primary_data/primary_data.csv",
    # set columns that can be null to nullable ints
    dtype={"body_part_2": "Int64", "diagnosis_2": "Int64"},
)
df.head(2)

In [ ]:
medical_terms = {
    "&": "and",
    "***": "",
    ">>": "clinical diagnosis",
    "@": "at",
    "+": "with",
    "?": "unknown if",
    "abd": "abdomen",
    "af": "accidental fall",
    "afib": "atrial fibrillation",
    "alf": "assisted living facility",
    "aki": "acute kidney injury",
    "am": "morning",
    "ams": "altered mental status",
    "bac": "blood alcohol content",
    "biba": "brought in by ambulance",
    "bwd": "backwards",
    "c/o": "complains of",
    "chi": "closed-head injury",
    "clsd": "closed",
    "cpk": "creatine phosphokinase",
    "cva": "cerebral vascular accident",
    "dn": "down",
    "dtr": "daughter",
    "dx": "clinical diagnosis",
    "ecf": "extended care facility",
    "er": "emergency room",
    "etoh": "ethyl alcohol",
    "eval": "evaluation",
    "fd": "found",
    "ft": "foot",
    "fx": "fracture",
    "fxs": "fractures",
    "fwd": "forwards",
    "glf": "ground level fall",
    "h/o": "history of",
    "hr": "hours",
    "htn": "hypertension",
    "hx": "history of",
    "inj": "injury",
    "inr": "international normalized ratio",
    "intox": "intoxication",
    "lac": "laceration",
    "loc": "loss of consciousness",
    "lt": "left",
    "mech": "mechanical",
    "mult": "multiple",
    "n h ": "nursing home",
    "nh": "nursing home",
    "p/w": "presents with",
    "pm": "afternoon",
    "pt": "patient",
    "pta": "prior to arrival",
    "pts": "patient's",
    "px": "physical examination", # not "procedure",
    "r/o": "rules out",
    "rt": "right",
    "s/p": "after",
    "sah": "subarachnoid hemorrhage",
    "sdh": "acute subdural hematoma",
    "sts": "sit to stand",
    "tr": "trauma",
    "uti": "urinary tract infection",
    "unwit'd": "unwitnessed",
    "w/o": "without",
    "w/": "with",
    "wks": "weeks"
}

def add_space_after(text, target=","):
    replaced_text = re.sub(r'({})(\S)'.format(re.escape(target)), r'{} \2'.format(target),text)
    return replaced_text

def add_space_before(text, target=","):
    replaced_text = re.sub(r'(\S)({})'.format(re.escape(target)), r'\1 {}'.format(target), text)
    return replaced_text

# cleanning
def clean_narrative(text):
    # lowercase everything
    if pd.isna(text):
        return "NA"
    else:     
        text = text.lower()
        
        # unglue DX
        regex_dx = r"([ˆ\W]*(dx)[ˆ\W]*)"
        text = re.sub(regex_dx, r". dx: ", text)
    
        # remove age and sex identifications
        ## regex to capture age and sex (not perfect but captures almost all of the cases)
        regex_age_sex = r"(\d+)\s*?(yof|yf|yo\s*female|yo\s*f|yom|ym|yo\s*male|yo\s*m)"
        age_sex_match = re.search(regex_age_sex, text)
    
        ## format age and sex
        if age_sex_match:
            age = age_sex_match.group(1)
            sex = age_sex_match.group(2)
            
            # probably not best practice but it works with this data
            if "f" in sex:
                #text = text.replace(age_sex_match.group(0), f"{age} years old female")
                text = text.replace(age_sex_match.group(0), f"patient")
            elif "m" in sex:
                #text = text.replace(age_sex_match.group(0), f"{age} years old male")
                text = text.replace(age_sex_match.group(0), f"patient")
                
        text = add_space_after(text, target=",") 
        text = add_space_after(text, target=";") 
        text = add_space_after(text, target=":") 
        text = add_space_before(text, target="--")  
        text = add_space_after(text, target="--")
        text = re.sub("-", " ", text)
        text = re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", text).strip()
        text = re.sub("\.", " ", text)
        
        # The below two are separated from the dictionary because of the '&' character conflict
        #   with the translate medical terms section use of '&'
        text = re.sub("t'd&f", "tripped and fell", text)
        text = re.sub("s'd&f", "slipped and fell",text)     
    
        
        # translate medical terms
        for term, replacement in medical_terms.items():
            if term == "@" or term == ">>" or term == "&" or term == "***" or term == "+" or term == "?":
                pattern = fr"({re.escape(term)})"
                text = re.sub(pattern, f" {replacement} ", text) # force spaces around replacement
                
            else:
                pattern = fr"(?<!-)\b({re.escape(term)})\b(?!-)"
                text = re.sub(pattern, replacement, text)
                            
        # This done after translate medical terms because some use a '/'
        text = add_space_before(text, target="/")  
        text = add_space_after(text, target="/")
        
        # remove extra white spaces
        text = re.sub(r'\s+', ' ', text)
    
        return text.upper()

In [ ]:
text = '72 YOF SLIPPED AND FELL ON THE FLOOR THIS AM>>L-3, L-4 FRACTURE, +LOC, RT RIB FRACTURES X 3.'
print("Original text:", text)
print("Clean text:", clean_narrative(text))

In [ ]:
df_narratives = df.iloc[:, 0:2]
df_narratives['narrative'] = df_narratives['narrative'].apply(lambda x: clean_narrative(x))

# Rename original narrative column
df_nn = df.rename(columns={"narrative": "narrative_original"})

df_final = pd.merge(df_narratives, df_nn, on="cpsc_case_number", how="left")

len(df_final)

In [ ]:
df_final.to_csv("corrected_narrative_primary.csv", index=False)

### replace numeric values with corresponding strings

In [ ]:
decoded_df = df_final.copy()

for col in mapping.keys():
    decoded_df[col] = decoded_df[col].map(mapping[col])

In [ ]:
decoded_df.head(3)

In [ ]:
# ensure mappings were applied correctly by checking that the number of missing values did not change
assert (decoded_df.isnull().sum() == df_final.isnull().sum()).all()

Add a column that reveals how many characters are in each narrative

In [ ]:
decoded_df['narrative_characters'] = decoded_df['narrative'].str.len()

In [ ]:
sns.kdeplot(decoded_df['narrative_characters'])
# plt.savefig("Narrative_character_count.pdf", format="pdf")

In [ ]:
sns.displot(decoded_df, x = 'narrative_characters', hue = 'sex')

In [ ]:
sns.displot(decoded_df, x = 'narrative_characters', hue = 'sex', kind = 'kde', fill = True)

In [ ]:
decoded_df.to_csv("decoded_primary_data_with_char_lens.csv")

Take a random sample for testing purposes

In [ ]:
random_sample = decoded_df.sample(n=5000, replace=False, random_state=42)

In [ ]:
random_sample_sub = random_sample.iloc[:,0:2]

In [ ]:
random_sample_sub.head(3)

## Model training

Import semi-manually labeled training data

In [ ]:
labeled_sets = pd.read_csv('secondary_data/labeled_training_data.csv')
labeled_sets.head(3)

Check how long the imported data frame is

In [ ]:
len(labeled_sets)

Check how many unique cases are contained in the data set

In [ ]:
key_list = labeled_sets.cpsc_case_number.unique()
len(key_list)

Randomly shuffle the key list in case there is some bias in the order of the entries

In [ ]:
np.random.seed(99)
np.random.shuffle(key_list, )
key_list

In [ ]:
nlp = spacy.blank("en")
# special_case = [{Token.ORTH: "CLOSED-HEAD"}]
# nlp.tokenizer.add_special_case("CLOSED-HEAD", special_case)

# keeping span token lengths to appropriately set config
token_lengths = []
label_list = []

docs=[] # this will hold the processed strings and spans

for case in key_list:
    # print(case)
    # print(k) # uncomment for troubleshooting

    temp_df = labeled_sets[labeled_sets.cpsc_case_number == case]
    nar = temp_df.iloc[0,1]
        
    doc = nlp(nar)
    
    if len(temp_df)==1:    
        if pd.isna(temp_df.iloc[0,2]):
            print("this entry has no label")
            doc.spans["sc"] = []
            docs.append(doc)
        else:     
            span_text = temp_df.iloc[0,2]
            temp_label = temp_df.iloc[0,3]   
            span_start_char = nar.find(span_text)
            span_end_char = span_start_char + len(span_text)
            
            # Finding the start and end tokens using character offsets
            start_token = None
            end_token = None
            for token in doc:
                if token.idx == span_start_char:
                     start_token = token.i
                if token.idx + len(token.text) == span_end_char:
                    end_token = token.i
                    break
                    
            if start_token is not None and end_token is not None:
                temp_start = start_token
                temp_end = end_token + 1
                
                doc.spans["sc"] = [Span(doc, temp_start, temp_end, temp_label)]
                
                token_lengths.append(temp_end - temp_start)
                label_list.append(temp_label)
            
                docs.append(doc)
            else:
                print(nar, "span=", span_text,"couldn't find tokens")
    else:
        print("temp_df has length > 1")
        span_list = []
        for ent in range(len(temp_df)):
            span_text = temp_df.iloc[ent,2]
            temp_label = temp_df.iloc[ent,3]
            span_start_char = nar.find(span_text)
            span_end_char = span_start_char + len(span_text)
            # print(span_start_char, span_end_char)

            # Finding the start and end tokens using character offsets
            start_token = None
            end_token = None
            for token in doc:
                if token.idx == span_start_char:
                     start_token = token.i
                if token.idx + len(token.text) == span_end_char:
                    end_token = token.i
                    break
            if start_token is not None and end_token is not None:
                temp_start = start_token
                temp_end = end_token + 1             
                span_list.append(Span(doc, temp_start, temp_end, temp_label))
                token_lengths.append(temp_end - temp_start)
                label_list.append(temp_label)
            else:
                print(nar, "span=",span_text, "couldn't find tokens")
        
        doc.spans["sc"] = span_list
        docs.append(doc)

In [ ]:
len(docs)

In [ ]:
for doc in docs[0:5]:
    print(doc, doc.spans)

In [ ]:
len(token_lengths)

Check on length of tokens in spans to set appropriate parameters in training config file

In [ ]:
print(np.min(token_lengths), np.max(token_lengths), np.median(token_lengths))

In [ ]:
np.quantile(token_lengths, q =[0.05,0.95])

Want to split the data into train and test such that a single case is not split between the two

In [ ]:
docs[1100]

In [ ]:
docs[1101]

Make training and test sets with the docs

In [ ]:
doc_bin = DocBin(docs=docs[0:1101])
doc_bin.to_disk("./train_falling.spacy")

doc_bin = DocBin(docs=docs[1101:])
doc_bin.to_disk("./dev_falling.spacy")

Initialize the training config file. This will get some manual adjustments.

In [ ]:
!python -m spacy init config configs/config_spancat_singlelabel.cfg --lang en --pipeline spancat_singlelabel --force

In [ ]:
!python -m spacy train configs/config_HP4.cfg --paths.train train_falling.spacy --paths.dev dev_falling.spacy --training.eval_frequency 100  --system.seed 99 --output spacy_falling_model4/

## Using model

In [ ]:
nlp_spancat = spacy.load("spacy_falling_model/model-best")

In [ ]:
nlp_spancat.pipeline

**Test model on a subset of the samples**

In [ ]:
cols = ['cpsc_case_number','text', 'span_text',  'span_label', ]
output_df = pd.DataFrame(columns=cols)

for row in random_sample_sub.iloc[1000:1050:,0:2].iterrows():
    cpsc = (row[1]['cpsc_case_number'])
    text = (row[1]['narrative'])
    doc = nlp_spancat(text)

    
    if len(doc.spans["sc"]) == 0:
        df2 = pd.DataFrame([[cpsc,text, "NA", "NA"]], columns= cols)
        # Append the new row to the DataFrame
        output_df = pd.concat([output_df, df2])
    else:
        for span in doc.spans["sc"]:
        # Create a new row as a dictionary
        
            df2 = pd.DataFrame([[cpsc,text, span.text, span.label_,]], columns=cols)
            # Append the new row to the DataFrame
            output_df = pd.concat([output_df, df2])

In [ ]:
output_df.head()

## Run model on all samples

In [ ]:
decoded_df.iloc[0:3:,0:2]

In [ ]:
for row in decoded_df.iloc[0:12:,0:2].iterrows():
    cpsc = (row[1]['cpsc_case_number'])
    text = (row[1]['narrative'])
    doc = nlp_spancat(text)
    print(doc.spans["sc"])

In [ ]:
# Create an empty DataFrame with column names
cols = ['cpsc_case_number','text', 'span_text', 'span_label']
output_df = pd.DataFrame(columns=cols)

for row in decoded_df.iloc[:,0:2].iterrows():
    cpsc = (row[1]['cpsc_case_number'])
    text = (row[1]['narrative'])
    doc = nlp_spancat(text)

    
    if len(doc.spans["sc"]) == 0:
        df2 = pd.DataFrame([[cpsc, text, "NA", "NA"]], columns=cols)
        # Append the new row to the DataFrame
        output_df = pd.concat([output_df, df2])
    else:
        for span in doc.spans["sc"]:
        # Create a new row as a dictionary
        
            df2 = pd.DataFrame([[cpsc, text, span.text, span.label_]], columns=cols)
            # Append the new row to the DataFrame
            output_df = pd.concat([output_df, df2])

In [ ]:
output_df.head()

In [ ]:
len(output_df)

In [ ]:
output_df.to_csv("predictions_falling_full_set5.csv")

In [ ]:
# Pivot the DataFrame
def tuple_aggregator(series):
    return tuple(series)

pivot_output_df = output_df.pivot_table(index=['cpsc_case_number', 'text'], columns='span_label', values='span_text', aggfunc=tuple_aggregator).reset_index()
pivot_output_df.columns.name = None 
pivot_output_df.head()

In [ ]:
# Make a copy of the original DataFrame
binary_df = pivot_output_df.copy()

# Update the DataFrame to have 0 for NaN and 1 for actual values
# We're excluding the first two columns ('A' and 'B') as they are index columns
binary_df.iloc[:, 2:] = binary_df.iloc[:, 2:].map(lambda x: 0 if x == 'NA' or pd.isna(x) else 1)
binary_df.head()


In [ ]:
binary_df.to_csv("predictions_full_set_binary_231003.csv")

In [ ]:
combo_df = pd.merge(binary_df, decoded_df, how="left", on="cpsc_case_number")
combo_df.columns

### Clean labels for plotting

In [ ]:
combo_df['diagnosis'] = combo_df['diagnosis'].str.replace(r'\d+ - ', '', regex=True)
print(combo_df['diagnosis'].unique())

In [ ]:
combo_df['diagnosis']= combo_df['diagnosis'].str.title()

In [ ]:
combo_df['disposition'] = combo_df['disposition'].str.replace(r'\d+ - ', '', regex=True)
print(combo_df['disposition'].unique())

In [ ]:
label_mapping = {'TREATED AND ADMITTED/HOSPITALIZED': 'Admitted', 
                 'TREATED/EXAMINED AND RELEASED': 'Released', 
                 'HELD FOR OBSERVATION': 'Held', 
                 'TREATED AND TRANSFERRED': 'Transferred',  
                 'LEFT WITHOUT BEING SEEN': 'Unseen', }
combo_df['disposition'] = combo_df['disposition'].map(label_mapping)
print(combo_df['disposition'].unique())

In [ ]:
sns.displot(combo_df, x = 'narrative_characters', hue = 'disposition', kind = 'kde')

In [ ]:
temp_df = pd.read_csv("predictions_falling_full_set5.csv")
output_df = temp_df.iloc[:,1:5]
output_df.head()

### Words within labeled spans

In [ ]:
SO_entries = output_df[output_df['span_label'] == 'SO']
len(SO_entries)

In [ ]:
SO_entries.head()

In [ ]:
def flatten_list(nested_list):
    return [item for sublist in nested_list for item in sublist]

In [ ]:
OBJ_entries = output_df[output_df['span_label'] == 'OBJ']

In [ ]:
list_remove_OBJ = ['TRIPPED', 'OVER', 'A', 'AN','ON', 'THE', 'HER', 
                   'OF', 'HIS', 'TRIPPING', 'FALL', 'FELL', 'SLIPPED', 'OWN', 'FEET']
OBJ_words = []
for i in range(len(OBJ_entries)):
    temp_entry = OBJ_entries.iloc[i,2]
    temp_text  = temp_entry.split(' ')
    out_temp = list(set(temp_text)-set(list_remove_OBJ))
    OBJ_words.append(out_temp) 

In [ ]:
OBJ_word_counts = collections.Counter(flatten_list(OBJ_words))
my_dict = dict(OBJ_word_counts)

rows = []
for key, value in my_dict.items():
    row = {'element': key, 'count': value}
    rows.append(row)
df = pd.DataFrame(rows).sort_values('count', ascending=False)

df.set_index('element', inplace=True)
OBJ_plot_subset = df.head(13)

In [ ]:
ax = OBJ_plot_subset.plot.barh(width = 0.8)
plt.ylabel('')
plt.xlabel('Count')
plt.title('Object causing fall')
plt.gca().get_legend().remove()
plt.gca().invert_yaxis()
plt.grid(axis='x', linestyle='--', linewidth=0.7, alpha=0.7)
plt.savefig("Object_causing.pdf", format="pdf", pad_inches = 0.5, bbox_inches = 'tight')

In [ ]:
list_remove = ['A', 'HEAD', 'HIT', 'THE', 'ON', 'HITTING', 'STRIKING', 
               'HER', 'OF', 'AGAINST', 'STRUCK', 'ONTO', 'HIS', 'INTO',
              'FACE','BACK', 'SIDE', 'CHEST', 'RIGHT', 'LEFT', 'EDGE', 
               'FOREHEAD', 'STAND','FRAME', 'AND', 'COFFEE', 'LANDED', 'ARM'
              ]
SO_words = []
for i in range(len(SO_entries)):
    temp_entry = SO_entries.iloc[i,2]
    temp_text  = temp_entry.split(' ')
    out_temp = list(set(temp_text)-set(list_remove))
    SO_words.append(out_temp)    

In [ ]:
sns.set_theme(style="white")
              
SO_word_counts = collections.Counter(flatten_list(SO_words))
my_dict = dict(SO_word_counts)

rows = []
for key, value in my_dict.items():
    row = {'element': key, 'count': value}
    rows.append(row)
df = pd.DataFrame(rows).sort_values('count', ascending=False)

df.set_index('element', inplace=True)
SO_plot_subset = df.head(15)
SO_plot_subset

In [ ]:
ax = SO_plot_subset.plot.barh(width = 0.8)
plt.ylabel('')
plt.xlabel('Count')
plt.title('Object struck')
plt.gca().get_legend().remove()
plt.gca().invert_yaxis()
plt.grid(axis='x', linestyle='--', linewidth=0.7, alpha=0.7)
plt.savefig("Object_struck.pdf", format="pdf", pad_inches = 0.5, bbox_inches = 'tight')

Gather rows for specific strike types

In [ ]:
object_involved = combo_df[combo_df['OBJ'] == 1]
len(object_involved)

In [ ]:
struck_object = combo_df[(combo_df['SO']==1) & (combo_df['SF']!=1) & (combo_df['SU']!=1)]
struck_floor = combo_df[(combo_df['SO']!=1) & (combo_df['SF']==1) & (combo_df['SU']!=1)]
struck_unknown = combo_df[(combo_df['SO']!=1) & (combo_df['SF']!=1) & (combo_df['SU']==1)]
print(len(struck_floor), len(struck_object), len(struck_unknown))

Bind the struck object and struck floor sets back together for plotting

In [ ]:
struck_object = struck_object.assign(struck = 'Object')
struck_floor = struck_floor.assign(struck = 'Floor')
striking_df = pd.concat([struck_object, struck_floor ], ignore_index=True) 
striking_df.head(3)

Plot disposition distribution for each category

In [ ]:
strike_disp = striking_df[['disposition', 'struck' ]]
strike_disp_counts = strike_disp.groupby(['struck', 'disposition']).size().reset_index(name='count')
temp = strike_disp_counts.pivot(columns = 'disposition', index = 'struck', values = 'count' )
temp.columns.name=''
col_order = ['Released', 'Admitted', 'Held', 'Transferred', 'Unseen']
temp2 = temp[col_order]

In [ ]:
# Create the figure
plt.figure(figsize=(8, 4))

#sns.set_palette('colorblind')
plt.style.use('tableau-colorblind10')

# Convert to relative proportions 
temp2.apply(lambda x: x*100/sum(x), axis = 1).plot(kind = 'bar', stacked = True, fontsize = 14, edgecolor='none')
plt.legend(bbox_to_anchor=(1, 1), loc=2) 
plt.ylabel("Percent", size = 16)
plt.xlabel("")
plt.savefig("Disposition_by_strike.pdf", format="pdf", pad_inches = 0.5, bbox_inches = 'tight')

In [ ]:
diag_cols_to_use = ['Internal Injury',
 'Fracture',
 'Laceration',
 'Contusions, Abr.',
 'Avulsion',
 'Hematoma',
 'Concussion',
 'Strain, Sprain',
 'Dislocation']

In [ ]:
striking_other = striking_df[striking_df['diagnosis'].isin(diag_cols_to_use) == False]

In [ ]:
other_diag_counts = striking_other.groupby(['struck', 'diagnosis']).size().reset_index(name='count').groupby(['struck']).sum('count')
other_diag_counts.reset_index()
other_diag_counts.rename(columns = {'count': 'Other'}, inplace = True)
other_diag_counts

In [ ]:
strike_diag = striking_df[['diagnosis', 'struck' ]]
strike_diag_sub = strike_diag[strike_diag['diagnosis'].isin(diag_cols_to_use)]
strike_diag_counts = strike_diag_sub.groupby(['struck', 'diagnosis']).size().reset_index(name='count')
temp = strike_diag_counts.pivot(columns = 'diagnosis', index = 'struck', values = 'count' )
temp.columns.name=''
temp_ordered = temp[diag_cols_to_use]
new_df = temp_ordered.merge(other_diag_counts, left_on = 'struck', right_on = 'struck')
new_df

In [ ]:
# Create the figure
plt.figure(figsize=(8, 4))
sns.set_style("whitegrid")
#sns.set_palette('deep')
plt.style.use('tableau-colorblind10')
new_df.apply(lambda x: x*100/sum(x), axis = 1).plot(kind = 'bar', stacked = True, fontsize = 14, edgecolor = 'none')
plt.legend(bbox_to_anchor=(1, 1), loc=2) 
plt.ylabel("Percent", size = 16)
plt.xlabel("")
plt.savefig("Diagnosis_by_strike.pdf", format="pdf", pad_inches = 0.5, bbox_inches = 'tight')

Look at frequencies of dispositon and diagnosis by strike type

In [ ]:
sf_disp = struck_floor['disposition'].value_counts()
sf_disp

In [ ]:
so_disp = struck_object['disposition'].value_counts()
print(so_disp["Released"], so_disp["Admitted"])

Use an odds ratio contigency table to determine if the striking the floor has a higher risk of hospitalization than striking an object.

In [ ]:
from scipy.stats.contingency import odds_ratio
res = odds_ratio([[sf_disp["Admitted"],so_disp["Admitted"]],[sf_disp["Released"],so_disp["Released"]]])
print(res.statistic, res.confidence_interval(confidence_level=0.95))

The odds of being admitted/hospitalized if a patient strikes the floor are 1.xx times (95% CI = 1.xx-1.xx) that of being admitted if a patient strikes an object.

In [ ]:
struck_unknown['disposition'].value_counts(normalize=True)

In [ ]:
so_diag = struck_object['diagnosis'].value_counts()

In [ ]:
sf_diag = struck_floor['diagnosis'].value_counts()

In [ ]:
sf_diag.sum() - sf_diag["Fracture"]

In [ ]:
res = odds_ratio([[sf_diag["Fracture"],so_diag["Fracture"]],[sf_diag.sum() - sf_diag["Fracture"], so_diag.sum() - so_diag["Fracture"]]])
print(res.statistic, res.confidence_interval(confidence_level=0.95))

In [ ]:
res = odds_ratio([[sf_diag["Laceration"],so_diag["Laceration"]],[sf_diag.sum() - sf_diag["Laceration"], so_diag.sum() - so_diag["Laceration"]]])
print(res.statistic, res.confidence_interval(confidence_level=0.95))

In [ ]:
struck_unknown['diagnosis'].value_counts(normalize=True)

In [ ]:
# another possible version of a stacke bar plot
# struck_floor.groupby('disposition')['diagnosis'].value_counts(normalize=True).unstack('diagnosis').plot.bar(stacked=True)